In [3]:
import numpy as np

In [4]:
class GDA_Classifier(object):  
    
    def __init__(self, epsilon = 0.001):
        self.epsilon = epsilon #small value to solve non-invertible matrix issue
    
    def fit(self,X,y):
        self.y = y
        
        self.miu = [np.sum(X[y==k],axis=0)/np.sum(y==k) for k in np.unique(y)]
        
        m = y.shape[0]
        phi = [1/(m)*np.sum(y==k) for k in np.unique(y)]
        self.phy = phi
        
        M = np.zeros_like(X)
        
        for k in np.unique(y):
        
            M[y==k] = self.miu[k]
            
        sigma =((X-M).T@(X-M)) / len(X)
        self.sig = sigma
        return self
    
    #compute probability of x|y
    def prob_xy(self,X):
        n = X.shape[0]
        
        probs=[]
        
        f1 = (2*np.pi)**(n/2)*((np.linalg.det(self.sig+self.epsilon*np.eye(n)))**0.5) 
        for k in np.unique(self.y):
            f2 = ((X - self.miu[k])@ (np.linalg.inv(self.sig+self.epsilon*np.eye(n))))@(X - self.miu[k]).T
            prob_x_y_k = (1/f1)*np.exp(- 0.5*f2)*self.phy[k]
            
            probs.append(prob_x_y_k)
        return probs
    
    #function to make the predictions
    def predict(self,X):
        
        predictions = []
        for i in range(len(X)):
            predictions.append(np.argmax(self.prob_xy(X[i])))
        return predictions
    